In [ ]:
import cv2
import numpy as np
import logging
import os
from collections import deque

def setup_logging(output_dir):
    log_file = os.path.join(output_dir, "motion_smoothing.log")
    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s - %(levelname)s - %(message)s",
        handlers=[
            logging.FileHandler(log_file),  # Log su file
            logging.StreamHandler()         # Log su console
        ]
    )
    logging.info("Logging configurato. Log salvati in: %s", log_file)

def add_transparent_overlay(frame, boxes, color=(0, 0, 255), alpha=0.3):
    """
    Disegna rettangoli rossi trasparenti sulle aree di movimento.
    """
    overlay = frame.copy()
    for (x, y, w, h) in boxes:
        cv2.rectangle(overlay, (x, y), (x + w, y + h), color, -1)
    cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)

def temporal_smoothing_flow(
    video_path, 
    output_dir, 
    flow_threshold=0.5,       # soglia di magnitudo per considerare un pixel in movimento
    alpha_fraction=0.2,       # % di frame su N in cui il pixel deve essere acceso
    window_size=30,           # N: lunghezza finestra temporale
    morph_kernel=3,           # dimensione kernel morfologico ridotto
    save_name="flow_smoothing_output.mp4",
    mask_save_name="movement_mask.mp4",  # Nome del video maschera binaria
    margin=10,                # padding in pixel
    scale_factor=0.5,         # riduzione di scala per il calcolo dell'Optical Flow
    skip_frames=1             # numero di frame da saltare (0 = nessuno)
):
    """
    Rileva il movimento usando Farneback Optical Flow e produce:
      - Un video con overlay (rettangoli rossi) = save_name
      - Un video con maschera binaria (0/255) = mask_save_name
    """
    logging.info("=== Inizio Optical Flow con Buffer Temporale ===")
    logging.info(f"Video input: {video_path}")
    logging.info(f"Finestra temporale: N={window_size}, soglia %={alpha_fraction}, scale_factor={scale_factor}")
    
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        logging.error("Impossibile aprire il video.")
        return

    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    logging.info(f"FPS: {fps} | Dimensioni: {width}x{height}")

    # VideoWriter per video con overlay
    fourcc_overlay = cv2.VideoWriter_fourcc(*'mp4v')
    output_path = os.path.join(output_dir, save_name)
    out_overlay = cv2.VideoWriter(output_path, fourcc_overlay, fps, (width, height))

    # VideoWriter per la maschera binaria (isColor=False)
    fourcc_mask = cv2.VideoWriter_fourcc(*'mp4v')
    mask_output_path = os.path.join(output_dir, mask_save_name)
    out_mask = cv2.VideoWriter(mask_output_path, fourcc_mask, fps, (width, height), isColor=False)

    ret, first_frame = cap.read()
    if not ret:
        logging.error("Non riesco a leggere il primo frame.")
        return

    # Converto il primo frame in scala di grigi e riduco per Optical Flow
    prev_gray_full = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
    prev_gray = cv2.resize(prev_gray_full, (0, 0), fx=scale_factor, fy=scale_factor)

    # Deque per maschere negli ultimi N frame
    mask_queue = deque(maxlen=window_size)
    frame_count = 1

    # Kernel morfologico
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (morph_kernel, morph_kernel))

    while True:
        ret, frame = cap.read()
        if not ret:
            logging.info("Fine del video.")
            break

        # Se vuoi saltare frame per velocità
        if skip_frames > 0 and frame_count % (skip_frames + 1) != 0:
            gray_full = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            gray = cv2.resize(gray_full, (0, 0), fx=scale_factor, fy=scale_factor)
            prev_gray = gray.copy()
            frame_count += 1
            continue

        gray_full = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.resize(gray_full, (0, 0), fx=scale_factor, fy=scale_factor)

        # Optical Flow
        flow = cv2.calcOpticalFlowFarneback(
            prev_gray, gray, None,
            0.5,    # scale
            2,      # levels
            9,      # winsize
            2,      # iterations
            5,      # poly_n
            1.2,    # poly_sigma
            0       # flags
        )

        # Magnitudo e Angolo
        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1], angleInDegrees=False)
        mask_current_resized = (mag > flow_threshold).astype(np.uint8) * 255
        # Risali alla risoluzione originale
        mask_current = cv2.resize(mask_current_resized, (width, height), interpolation=cv2.INTER_NEAREST)

        # Aggiungi la maschera alla coda
        mask_queue.append(mask_current)

        # Costruisci la maschera "media" con smoothing temporale
        if frame_count <= window_size:
            cumulative_mask = np.sum(np.array(mask_queue), axis=0)
        else:
            cumulative_mask = np.sum(np.array(mask_queue), axis=0)

        mask_smoothed = (cumulative_mask >= (alpha_fraction * len(mask_queue) * 255)).astype(np.uint8) * 255

        # Operazioni morfologiche
        mask_smoothed = cv2.morphologyEx(mask_smoothed, cv2.MORPH_CLOSE, kernel)
        mask_smoothed = cv2.morphologyEx(mask_smoothed, cv2.MORPH_OPEN, kernel)

        # Trova contorni e crea bounding box
        contours, _ = cv2.findContours(mask_smoothed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        boxes = []
        for cnt in contours:
            x, y, w, h = cv2.boundingRect(cnt)
            x_padded = max(0, x - margin)
            y_padded = max(0, y - margin)
            w_padded = min(w + 2*margin, width - x_padded)
            h_padded = min(h + 2*margin, height - y_padded)

            if w_padded > 5 and h_padded > 5:
                boxes.append((x_padded, y_padded, w_padded, h_padded))

        if frame_count % 10 == 0:
            logging.info(f"Frame {frame_count} - {len(boxes)} aree di movimento")

        # Disegno overlay
        if boxes:
            add_transparent_overlay(frame, boxes, (0, 0, 255), alpha=0.3)

        # Scrivi il frame con overlay
        out_overlay.write(frame)
        # Scrivi la maschera binaria
        out_mask.write(mask_smoothed)

        cv2.imshow("Temporal Smoothing Flow", frame)

        # Aggiorna
        prev_gray = gray.copy()
        frame_count += 1

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out_overlay.release()
    out_mask.release()
    cv2.destroyAllWindows()
    logging.info(f"Video overlay salvato in: {output_path}")
    logging.info(f"Video maschera salvato in: {mask_output_path}")

def main():
    video_path = "../Dataset/input/test2.mp4"
    output_dir = "../Dataset/output/"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    setup_logging(output_dir)
    
    temporal_smoothing_flow(
        video_path=video_path,
        output_dir=output_dir,
        flow_threshold=0.5,
        alpha_fraction=0.2,
        window_size=30,
        morph_kernel=3,
        save_name="flow_smoothing_output.mp4",
        mask_save_name="movement_mask.mp4",
        margin=10,
        scale_factor=0.5,
        skip_frames=1
    )

if __name__ == "__main__":
    main()

In [ ]:
import cv2
import numpy as np
import logging
import os

def setup_logging(output_dir):
    log_file = os.path.join(output_dir, "compression.log")
    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s - %(levelname)s - %(message)s",
        handlers=[
            logging.FileHandler(log_file),  # Log su file
            logging.StreamHandler()         # Log su console
        ]
    )
    logging.info("Logging configurato. Log salvati in: %s", log_file)

# Esempio di tabella di quantizzazione molto forte
QTY_strong = np.full((8, 8), 500, dtype=np.float32)

def compress_block_8x8(block, Q_table):
    """
    Esegue DCT su blocco 8x8 e applica quantizzazione Q_table
    """
    dct_block = cv2.dct(np.float32(block) - 128.0)
    quant_block = np.round(dct_block / Q_table)
    return quant_block

def decompress_block_8x8(quant_block, Q_table):
    """
    Dequantizza e poi esegue IDCT su blocco 8x8
    """
    dequant_block = quant_block * Q_table
    idct_block = cv2.idct(dequant_block) + 128.0
    return np.uint8(np.clip(idct_block, 0, 255))

def compress_frame_with_motion_mask(frame_bgr, mask, blockSize=8):
    """
    - frame_bgr: frame BGR
    - mask: maschera binaria (0=statico, 255=movimento)
    - blockSize=8
      Ritorna frame compresso differenziato:
        - zone in movimento => nessuna compressione (blocco originale)
        - zone statiche => quantizzazione aggressiva
    """
    H, W, _ = frame_bgr.shape
    # Converti in YCrCb
    frame_ycrcb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2YCrCb)
    Y, Cr, Cb = cv2.split(frame_ycrcb)

    Y_recon = np.zeros_like(Y, dtype=np.uint8)

    for by in range(0, H, blockSize):
        for bx in range(0, W, blockSize):
            y_end = min(by+blockSize, H)
            x_end = min(bx+blockSize, W)
            blockY = Y[by:y_end, bx:x_end]
            blockMask = mask[by:y_end, bx:x_end]

            # se >20% pixel bianchi => blocco in movimento
            motion_ratio = (blockMask > 0).mean()
            if motion_ratio > 0.2:
                # Copia il blocco senza compressione
                recon_crop = blockY.copy()
            else:
                # Applica quantizzazione forte
                paddedY = np.zeros((8,8), dtype=np.float32)
                paddedY[:(y_end-by), :(x_end-bx)] = blockY
                quant_block = compress_block_8x8(paddedY, QTY_strong)
                recon_block = decompress_block_8x8(quant_block, QTY_strong)
                recon_crop = recon_block[:(y_end-by), :(x_end-bx)]

            Y_recon[by:y_end, bx:x_end] = recon_crop

    recon_ycrcb = cv2.merge([Y_recon, Cr, Cb])
    recon_bgr = cv2.cvtColor(recon_ycrcb, cv2.COLOR_YCrCb2BGR)
    return recon_bgr

def main_compress_with_motion(input_video, motionMask_video, output_video="COMPRESSION_output.mp4"):
    """
    Legge video originale e video maschera binaria,
    e crea un video compresso differenziato con QTY_strong nelle zone statiche,
    nessuna compressione nelle zone di movimento.
    """
    cap_in = cv2.VideoCapture(input_video)
    cap_mask = cv2.VideoCapture(motionMask_video)
    if not cap_in.isOpened() or not cap_mask.isOpened():
        print("Impossibile aprire uno dei due video.")
        return

    fps = cap_in.get(cv2.CAP_PROP_FPS)
    width = int(cap_in.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap_in.get(cv2.CAP_PROP_FRAME_HEIGHT))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

    frame_count = 0
    while True:
        ret_in, frame_in = cap_in.read()
        ret_mask, frame_mask = cap_mask.read()
        if not ret_in or not ret_mask:
            break
        frame_count += 1

        # Se la maschera è a 3 canali, convertila in GRAY
        if len(frame_mask.shape) == 3:
            gray_mask = cv2.cvtColor(frame_mask, cv2.COLOR_BGR2GRAY)
        else:
            gray_mask = frame_mask

        # comprimi
        compressed_frame = compress_frame_with_motion_mask(frame_in, gray_mask, blockSize=8)
        out.write(compressed_frame)

        if frame_count % 10 == 0:
            print(f"Frame {frame_count} processato.")

    cap_in.release()
    cap_mask.release()
    out.release()
    print(f"Video compresso salvato in: {output_video}")

def main():
    logging.basicConfig(level=logging.INFO)
    video_originale = "../Dataset/input/test2.mp4"
    video_maschera = "../Dataset/output/movement_mask.mp4"  # generato dal Blocco 1
    video_compresso = "../Dataset/output/COMPRESSION_test2.mp4"

    main_compress_with_motion(
        input_video=video_originale,
        motionMask_video=video_maschera,
        output_video=video_compresso
    )

if __name__ == "__main__":
    main()